In [1]:
default_headers = {
          'Content-Type': 'application/json',
          'Access-Control-Allow-Origin': '*'
        }

In [2]:
from IPython.display import display_javascript, display_html, display
import uuid

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 150px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [3]:
import requests, json

In [4]:
engine = "http://jasper:8181"

In [5]:
e = requests.get(engine + "/jasper").json()

In [6]:
RenderJSON(e)

In [7]:
l = requests.get(engine + "/jasper/streams").json()

In [8]:
RenderJSON(l)

In [9]:
qs = requests.get(engine + "/jasper/queries").json()

In [10]:
RenderJSON(qs)

In [13]:
redstream = requests.get("http://redstream:1255/red")

In [14]:
RenderJSON(redstream.json())

In [75]:
body = {
    'id':'redcount',
    'tbox': 'https://raw.githubusercontent.com/riccardotommasini/wasp-jasper/master/colors.owl',
    'body': '''
        SELECT (COUNT(?r) as ?numRed)
        FROM NAMED WINDOW <rw> ON <http://streamhub:9292/streamhub/streams/red> [RANGE PT15S STEP PT5S]
        WHERE {
                WINDOW ?rw { ?r a <http://www.streamreasoning.org/ontologies/2018/9/colors#Red> .}
            }

    '''
}
    
data = json.dumps(body)

RenderJSON(body)

In [76]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers)

In [77]:
q.text

'{\n  "@id" : "queries/redcount",\n  "@type" : "vprov:Task",\n  "prov:generated" : {\n    "@id" : "streams/redcount"\n  },\n  "prov:uses" : {\n    "@id" : "http://streamhub:9292/streamhub/streams/red"\n  },\n  "@context" : {\n    "@base" : "http://jasper:8181/jasper/",\n    "rdf" : "http://www.w3.org/1999/02/22-rdf-syntax-ns#",\n    "vsd" : "http://w3id.org/rsp/vocals-sd#",\n    "vocals" : "http://w3id.org/rsp/vocals#",\n    "xsd" : "http://www.w3.org/2001/XMLSchema#",\n    "format" : "http://www.w3.org/ns/formats/",\n    "rdfs" : "http://www.w3.org/2000/01/rdf-schema#",\n    "vprov" : "http://w3id.org/rsp/vocals-prov#",\n    "dcat" : "http://www.w3.org/ns/dcat#",\n    "prov" : "http://www.w3.org/ns/prov#"\n  }\n}'

In [78]:
RenderJSON(q.json())

In [79]:
o = requests.post(engine + "/jasper/observers/redcount", data = json.dumps({'protocol':"HTTP", "retention":5}) , headers=default_headers).json()

In [80]:
os = requests.get(engine + "/jasper/observers/redcount").json()

In [81]:
RenderJSON(o)

In [82]:
RenderJSON(os)

In [43]:
a = requests.get("http://jasper:8182/jasper/streams/redcount/observers/349655421")

In [44]:
a

<Response [200]>

In [83]:
o = requests.post(engine + "/jasper/observers/redcount", data = json.dumps({'protocol':"WEBSOCKET"}) , headers=default_headers).json()

In [84]:
RenderJSON(o)